<a href="https://colab.research.google.com/github/julianVelandia/FinetuningLLMPreQuantizationGradeWorksUNALDatasetInstruct/blob/master/FinetuningPreQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/user-attachments/assets/a200c62f-b3d0-4c44-a956-f19f268fc543" alt="image" width="400"/>


In [ ]:
!pip install datasets
!pip install peft transformers torch
!pip install -U bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

REINICIAR EL COLAB POR ESTE ERROR: https://discuss.huggingface.co/t/importerror-in-bitsandbytes-with-accelerate/80060

In [ ]:
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model
import torch
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
DATASET_NAME = "JulianVelandia/unal-repository-dataset-train-instruct"
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
OUTPUT_DIR = "/content/drive/MyDrive/Llama-3.2-1B-unal-instruct-q-ft"

LORA_RANK = 2
LORA_ALPHA = 8
LORA_DROPOUT = 0.2
HUGGINGFACE_TOKEN = ":)"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
from huggingface_hub import login

login(token=HUGGINGFACE_TOKEN)

dataset = load_dataset(DATASET_NAME, split="train")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/875 [00:00<?, ?B/s]

finetuning_dataset.json:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12525 [00:00<?, ? examples/s]

In [ ]:
for idx, row in enumerate(dataset):
    print(f"Fila {idx + 1}: Tipo - {type(row)}, Contenido - {repr(row)}")
    if idx == 4:
        break


dataset_info = {
    "Tipo": type(dataset),
    "Tamaño": len(dataset),
    "Características": dataset.column_names,
}
print(dataset_info)

Fila 1: Tipo - <class 'dict'>, Contenido - {'prompt': '¿Qué tipo de diseño de estudio se está utilizando para investigar la enfermedad nosocomial NAVM en el hospital?', 'completion': 'Diseño anidado en cohorte.', 'fragment': 'con diagnostico confirmado de NAVM. Este comité multidisciplinario, se encuentra funcionando desde el 15 de abril del 2016 y usa criterios internacionalmente aceptados para el diagnóstico de NAVM, además la NAVM como enfermedad nosocomial hace parte de los reportes oficiales de vigilancia epidemiológica que el hospital debe informar al Instituto Nacional de Salud (INS) y que se llenan de forma rutinaria en fichas epidemiológicas según lo establecido por el Ministerio de la Protección social en Colombia desde el año 2016. Todos los pacientes con diagnóstico de NAVM serán incluidos como casos verificando la presencia de criterios de inclusión y exclusión, los controles se tomarán de forma aleatoria (4 por cada caso) al momento de la aparición temporal del caso (dise

In [ ]:
!nvidia-smi
!nvcc --version


Tue Mar 11 03:39:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from bitsandbytes import __version__
print("Versión de bitsandbytes instalada:", __version__)

bnb_config = BitsAndBytesConfig(load_in_4bit=True)
token = HUGGINGFACE_TOKEN

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))

lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

Versión de bitsandbytes instalada: 0.45.3


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples["prompt"], truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(examples["completion"], truncation=True, padding="max_length", max_length=512)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["fragment"])

print("\nDataset tokenizado:")
print(tokenized_dataset)

Map:   0%|          | 0/12525 [00:00<?, ? examples/s]


Dataset tokenizado:
Dataset({
    features: ['prompt', 'completion', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12525
})


In [ ]:
from transformers import TrainerCallback

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=250,
    save_total_limit=2,
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_dir=f"{OUTPUT_DIR}/logs",
    fp16=True,
    push_to_hub=False
)
last_checkpoint = None
if os.path.exists(OUTPUT_DIR):
    checkpoints = [d for d in os.listdir(OUTPUT_DIR) if d.startswith("checkpoint")]
    if checkpoints:
        last_checkpoint = os.path.join(OUTPUT_DIR, max(checkpoints, key=lambda x: int(x.split('-')[-1])))
if last_checkpoint:
    print(f"Resumiendo desde el último checkpoint: {last_checkpoint}")
else:
    print("No se encontró un checkpoint previo. Iniciando desde cero.")


class SaveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        print(f"Checkpoint guardado en {args.output_dir} en el paso {state.global_step}")
        checkpoints = [d for d in os.listdir(args.output_dir) if d.startswith("checkpoint")]
        print(f"Checkpoints actuales: {checkpoints}")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    callbacks=[SaveCheckpointCallback()]
)



Resumiendo desde el último checkpoint: /content/drive/MyDrive/Llama-3.2-1B-unal-instruct-q-ft/checkpoint-4695


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-9edbd6f0b3d1>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print("Entrenando")
if last_checkpoint:
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    trainer.train()

Entrenando


/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jvelandiag (julian-unal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [ ]:
import os
print(os.listdir(OUTPUT_DIR))


['logs', 'checkpoint-4500', 'checkpoint-4695']


In [ ]:
from peft import PeftModel
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    device_map="auto"
)

model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/Llama-3.2-1B-unal-instruct-q-ft/checkpoint-4695")
model = model.merge_and_unload()

model.save_pretrained("/content/merged_llama")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.save_pretrained("/content/merged_llama")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  


Saving checkpoint shards:   0%|          | 0/1 [00:00<?, ?it/s]

('/content/merged_llama/tokenizer_config.json',
 '/content/merged_llama/special_tokens_map.json',
 '/content/merged_llama/tokenizer.json')

In [ ]:
!mkdir -p /content/gguf_model


In [ ]:
!rm -rf llama.cpp
!git clone https://github.com/ggerganov/llama.cpp

!pip install -r llama.cpp/requirements.txt

%cd llama.cpp
!mkdir build
%cd build
!cmake ..
!cmake --build . --config Release
%cd ../..

!mkdir -p /content/gguf_model

!python3 llama.cpp/convert_hf_to_gguf.py /content/merged_llama --outfile /content/gguf_model/model-f16.gguf --outtype f16

!llama.cpp/build/bin/llama-quantize /content/gguf_model/model-f16.gguf /content/gguf_model/model-q4_k_m.gguf Q4_K_M

!ls -lh /content/gguf_model/


Cloning into 'llama.cpp'...
remote: Enumerating objects: 45841, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 45841 (delta 9), reused 6 (delta 6), pack-reused 45829 (from 2)
Receiving objects: 100% (45841/45841), 96.01 MiB | 11.02 MiB/s, done.
Resolving deltas: 100% (33169/33169), done.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
/content/llama.cpp
/content/llama.cpp/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler

In [ ]:
!ls gguf_model/

model-f16.gguf	model-q4_k_m.gguf


In [ ]:
from huggingface_hub import HfApi

repo_name = "JulianVelandia/Llama-3.2-1B-unal-instruct-q-ft-gguf"
api = HfApi()

api.create_repo(repo_name, exist_ok=True)

api.upload_folder(
    folder_path="/content/merged_llama",
    repo_id=repo_name,
    repo_type="model"
)

api.upload_folder(
    folder_path="/content/gguf_model",
    repo_id=repo_name,
    repo_type="model"
)


model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-f16.gguf:   0%|          | 0.00/2.48G [00:00<?, ?B/s]

model-q4_k_m.gguf:   0%|          | 0.00/808M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/JulianVelandia/Llama-3.2-1B-unal-instruct-q-ft-gguf/commit/c3ddde45a6989eebbab07a7f3603db020bb3bdce', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c3ddde45a6989eebbab07a7f3603db020bb3bdce', pr_url=None, repo_url=RepoUrl('https://huggingface.co/JulianVelandia/Llama-3.2-1B-unal-instruct-q-ft-gguf', endpoint='https://huggingface.co', repo_type='model', repo_id='JulianVelandia/Llama-3.2-1B-unal-instruct-q-ft-gguf'), pr_revision=None, pr_num=None)